In [1]:
from detectors import *
import numpy as np

In [2]:
data_file = '../data/data-5-100.csv'
label_file = '../data/label-5-100.csv'

X = np.loadtxt(data_file, delimiter=",")
y = np.loadtxt(label_file, delimiter=",")

In [3]:
print("X shape ", X.shape, " y shape ", y.shape)

X shape  (500, 5)  y shape  (500,)


In [4]:
from sklearn.linear_model import LinearRegression

X_train, y_train = X[:50, :], y[:50]

model = LinearRegression()
model = model.fit(X_train, y_train)

In [5]:
def scorer(pred, label):
    return np.sum((pred - label) ** 2)

In [8]:
# d1 = RDDM()
d2 = WinRDDM(win_len=50 ,dtc_bd=3)

for i in range(500):
    next_sample = X[i, :]
    next_label = y[i]
    pred = model.predict([next_sample])
    error = scorer(pred, next_label)
       
    d2_out = d2.predict(next_sample, error)
    if d2_out:
        print("%d) d2 detection !" % i)
        print(d2_out)
        break


 avg :  1.4195719684024225e-26  std :  2.08338931855e-26
min avg :  None  min std :  None
 avg :  1.4403978963002572e-26  std :  2.08721566972e-26
min avg :  1.4195719684024225e-26  min std :  2.08338931855e-26
 avg :  1.2458206208620634e-26  std :  1.7229184458e-26
min avg :  1.4195719684024225e-26  min std :  2.08338931855e-26
 avg :  1.2465274402331413e-26  std :  1.72243303037e-26
min avg :  1.2458206208620634e-26  min std :  1.7229184458e-26
 avg :  1.2682368923448236e-26  std :  1.73634127238e-26
min avg :  1.2458206208620634e-26  min std :  1.7229184458e-26
 avg :  1.2553121952736825e-26  std :  1.74271530507e-26
min avg :  1.2458206208620634e-26  min std :  1.7229184458e-26
 avg :  1.1653441867550366e-26  std :  1.68580845238e-26
min avg :  1.2458206208620634e-26  min std :  1.7229184458e-26
 avg :  1.1447454507979055e-26  std :  1.68153288067e-26
min avg :  1.1653441867550366e-26  min std :  1.68580845238e-26
 avg :  1.1188960566556234e-26  std :  1.68898106307e-26
min avg :  

In [10]:
d2.get_warning_params()

(0, [])